In [1]:
class Empty(Exception):
    pass

In [2]:
class _DLListBase:
    """A base class providing a doubly linked list representation."""

    class _Node:
        __slots__ = "_element", "_prev", "_next"

        def __init__(self, element, prev_node, next_node):
            self._element = element
            self._next = next_node
            self._prev = prev_node

    def __init__(self):
        self._size = 0
        self._sentinel_front = self._Node(None, None, None)
        self._sentinel_back = self._Node(None, None, None)
        self._sentinel_front._next = self._sentinel_back
        self._sentinel_back._prev = self._sentinel_front

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def _insert_between(self, e, prev_node, next_node):
        "Add element e between two existing nodes."
        new_node = self._Node(e, prev_node, next_node)
        prev_node._next = new_node
        next_node._prev = new_node
        self._size += 1
        return new_node

    def _delete_node(self, node):
        "Delete nonsentinel node and return its element."
        prev_node = node._prev
        next_node = node._next
        e = node._element
        prev_node._next = next_node
        next_node._prev = prev_node
        self._size -= 1
        node._prev = node._next = node._element = None
        return e

<img src="dllist.png">

In [3]:
class DLList(_DLListBase):
    """List of objects using Doubly Linked List data structure."""

    def __getitem__(self, index):
        if self._size == 0:
            raise Empty("list is empty")
        elif not 0 <= index < self._size:
            raise IndexError("index out of range")
        position = index
        current_node = self._sentinel_front
        while position >= 0:
            current_node = current_node._next
            position -= 1
        return current_node._element

    def __setitem__(self, index, value):
        if not 0 <= index < self._size:
            raise IndexError("index out of range")
        position = index
        current_node = self._sentinel_front
        while position >= 0:
            current_node = current_node._next
            position -= 1
        current_node._element = value

    def add_first(self, element):
        self._insert_between(element, self._sentinel_front, self._sentinel_front._next)

    def add_last(self, element):
        self._insert_between(element, self._sentinel_back._prev, self._sentinel_back)

    def get_first(self):
        if self._size == 0:
            raise Empty("List is empty.")
        return self._sentinel_front._next._element

    def get_last(self):
        if self._size == 0:
            raise Empty("List is empty.")
        return self._sentinel_back._prev._element

    def insert(self, e, position):
        if position == 0:
            self.add_first(e)
        elif position >= self._size:
            self.add_last(e)
        else:
            current_node = self._sentinel_front
            while position > 0:
                current_node = current_node._next
                position -= 1
            self._insert_between(e, current_node, current_node._next)

    def remove_first(self):
        if self._size == 0:
            raise Empty("list is empty.")
        _ = self._delete_node(self._sentinel_front._next)

    def remove_last(self):
        if self._size == 0:
            raise Empty("list is empty.")
        _ = self._delete_node(self._sentinel_back._prev)

    def remove(self, e):
        if self._size == 0:
            raise Empty("list is empty.")
        current_node = self._sentinel_front._next
        while current_node:
            if current_node._element == e:
                _ = self._delete_node(current_node)
                return
            current_node = current_node._next
        raise ValueError(f"{e} not in list")

    def __contains__(self, e):
        if self._size == 0:
            raise Empty("list is empty.")
        current_node = self._sentinel_front._next
        while current_node:
            if current_node._element == e:
                return True
            current_node = current_node._next
        return False

    def count(self, e):
        if self._size == 0:
            raise Empty("list is empty.")
        current_node = self._sentinel_front._next
        total = 0
        while current_node:
            if current_node._element == e:
                total += 1
            current_node = current_node._next
        return total

    def reverse(self):
        if self._size <= 1:
            return
        current_node = self._sentinel_front
        while current_node:
            tmp = current_node._prev
            current_node._prev = current_node._next
            current_node._next = tmp
            current_node = current_node._prev
        self._sentinel_front, self._sentinel_back = self._sentinel_back, self._sentinel_front

    def __iter__(self):
        self._current_node = self._sentinel_front._next
        return self

    def __next__(self):
        while not self._current_node._next:
            raise StopIteration()
        e = self._current_node._element
        self._current_node = self._current_node._next
        return e

In [4]:
l = DLList()
l.add_first(10)
l[0]

10

In [5]:
l[10]

IndexError: index out of range

In [6]:
l.add_first(100)
l.add_first(1000)
l[0], l[1], l[2]

(1000, 100, 10)

In [7]:
l.add_last(0)
len(l)

4

In [8]:
for e in l:
    print(e)

1000
100
10
0


In [9]:
len(l)

4

In [10]:
l.insert(4, 10000)

In [11]:
l.remove_first()
for e in l:
    print(e)

100
10
0
4


In [12]:
l.remove_last()

In [13]:
l.insert(-10, 1)

In [14]:
for e in l:
    print(e)

100
-10
10
0


In [15]:
l.remove(100)

In [16]:
for e in l:
    print(e)

-10
10
0


In [17]:
l.remove(100)

ValueError: 100 not in list

In [18]:
len(l)

3

In [19]:
l[0] = 1_000_000
l[0]

1000000

In [20]:
l.add_last(10)
l.add_last(-10)
l.count(10)

2

In [21]:
10 in l

True

In [22]:
0 in l

True

In [23]:
it = iter(l)
next(it)

1000000

In [24]:
[e for e in l]

[1000000, 10, 0, 10, -10]

In [25]:
l.reverse()

In [26]:
[e for e in l]

[-10, 10, 0, 10, 1000000]

In [27]:
l[1] = 100
l.insert(3, 1000)

In [28]:
[e for e in l]

[-10, 100, 0, 10, 1000000, 3]

<img src="dllist-circular.png">

In [29]:
class _CircularDLListBase(_DLListBase):
    """A base class providing a circular doubly linked list representation."""

    def __init__(self):
        self._size = 0
        self._sentinel = self._Node(None, None, None)
        self._sentinel._prev = self._sentinel
        self._sentinel._next = self._sentinel

In [30]:
class CircularDLList(_CircularDLListBase):
    """List of objects using Doubly Linked List data structure."""

    def __getitem__(self, index):
        if self._size == 0:
            raise Empty("list is empty")
        elif not 0 <= index < self._size:
            raise IndexError("index out of range")
        position = index
        current_node = self._sentinel
        while position >= 0:
            current_node = current_node._next
            position -= 1
        return current_node._element

    def __setitem__(self, index, value):
        if not 0 <= index < self._size:
            raise IndexError("index out of range")
        position = index
        current_node = self._sentinel
        while position >= 0:
            current_node = current_node._next
            position -= 1
        current_node._element = value

    def add_first(self, element):
        self._insert_between(element, self._sentinel, self._sentinel._next)

    def add_last(self, element):
        self._insert_between(element, self._sentinel._prev, self._sentinel)

    def get_first(self):
        if self._size == 0:
            raise Empty("List is empty.")
        return self._sentinel._next._element

    def get_last(self):
        if self._size == 0:
            raise Empty("List is empty.")
        return self._sentinel._prev._element

    def insert(self, e, position):
        if position == 0:
            self.add_first(e)
        elif position >= self._size:
            self.add_last(e)
        else:
            current_node = self._sentinel
            while position > 0:
                current_node = current_node._next
                position -= 1
            self._insert_between(e, current_node, current_node._next)

    def remove_first(self):
        if self._size == 0:
            raise Empty("list is empty.")
        _ = self._delete_node(self._sentinel._next)

    def remove_last(self):
        if self._size == 0:
            raise Empty("list is empty.")
        _ = self._delete_node(self._sentinel._prev)

    def remove(self, e):
        if self._size == 0:
            raise Empty("list is empty.")
        current_node = self._sentinel._next
        position = 0
        while position < self._size:
            if current_node._element == e:
                _ = self._delete_node(current_node)
                return
            current_node = current_node._next
            position += 1
        raise ValueError(f"{e} not in list")

    def __contains__(self, e):
        if self._size == 0:
            raise Empty("list is empty.")
        current_node = self._sentinel._next
        while current_node is not self._sentinel:
            if current_node._element == e:
                return True
            current_node = current_node._next
        return False

    def count(self, e):
        if self._size == 0:
            raise Empty("list is empty.")
        current_node = self._sentinel._next
        total = 0
        while current_node is not self._sentinel:
            if current_node._element == e:
                total += 1
            current_node = current_node._next
        return total

    def reverse(self):
        if self._size <= 1:
            return
        start_node = current_node = self._sentinel
        position = 0
        while position <= self._size:
            tmp = current_node._prev
            current_node._prev = current_node._next
            current_node._next = tmp
            current_node = current_node._prev
            position += 1

    def __iter__(self):
        self._current_node = self._sentinel._next
        self._counter = 0
        return self

    def __next__(self):
        if self._counter == self._size :
            raise StopIteration()
        e = self._current_node._element
        self._current_node = self._current_node._next
        self._counter += 1
        return e

In [31]:
l = CircularDLList()
l.add_first(10)
l[0]

10

In [32]:
l[10]

IndexError: index out of range

In [33]:
l.add_first(100)
l.add_first(1000)
l[0], l[1], l[2]

(1000, 100, 10)

In [34]:
1000 in l, -1 in l

(True, False)

In [35]:
l.add_last(0)
len(l)

4

In [36]:
l[0], l[1], l[2], l[3]

(1000, 100, 10, 0)

In [37]:
for e in l:
    print(e)

1000
100
10
0


In [38]:
len(l)

4

In [39]:
l.insert(4, 10000)

In [40]:
l.remove_first()
for e in l:
    print(e)

100
10
0
4


In [41]:
l.insert(4, 1)

In [42]:
for e in l:
    print(e)

100
4
10
0
4


In [43]:
for e in l:
    print(e)

100
4
10
0
4


In [44]:
l.remove_last()

In [45]:
for e in l:
    print(e)

100
4
10
0


In [46]:
l.remove(100)

In [47]:
for e in l:
    print(e)

4
10
0


In [48]:
l.remove(100)

ValueError: 100 not in list

In [49]:
len(l)

3

In [50]:
l[0] = 1_000_000
l[0]

1000000

In [51]:
l.add_last(10)
l.add_last(-10)
l.count(10)

2

In [52]:
10 in l

True

In [53]:
0 in l

True

In [54]:
it = iter(l)
next(it)

1000000

In [55]:
[e for e in l]

[1000000, 10, 0, 10, -10]

In [56]:
l.reverse()

In [57]:
[e for e in l]

[-10, 10, 0, 10, 1000000]

In [58]:
l[1] = 100
l.insert(3, 1000)

In [59]:
[e for e in l]

[-10, 100, 0, 10, 1000000, 3]